## 텍스트 분류 8529data+twitter+remove(userdic=stopwords+missprint)+(Word2Vec?)

In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series
import re
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [3]:
df = pd.read_csv('new_data_8529.csv', sep='\t', encoding='UTF-8')

In [4]:
userdic = pd.read_csv('usedic.csv', sep='\t', encoding='UTF-8')

In [5]:
userdic['User Dic.'][0:1]

0    가니쉬
Name: User Dic., dtype: object

In [6]:
userdic = userdic.rename(columns = {'User Dic.':'standard'})
userdic = userdic.rename(columns = {'동의어':'missprint'})
userdic = userdic.rename(columns = {'불용단어':'stops'})

In [7]:
userdic = userdic.fillna('')

In [8]:
userdic[:5]

stops standard missprint
0    결과      가니쉬       가니시
1    공임       가스        개스
2    광주       가죽          
3    넥쏘       간격        간극
4    니로       간격         갭

In [9]:
userdic.shape

(656, 3)

In [10]:
userdic = userdic.astype(str).copy()

In [11]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [12]:
df.head(3)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---  
2  X---

In [13]:
df.shape

(8529, 17)

# 분류현황

In [14]:
category_count = df['대상 선정'].value_counts()
category_count

X    5517
O    3012
Name: 대상 선정, dtype: int64

In [15]:
category_count = df['구분'].value_counts()
category_count

기능(커넥터)    1601
실내부품이음     1050
하체이음        361
Name: 구분, dtype: int64

In [16]:
category_count = df['책임구'].value_counts()
category_count

구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [17]:
category_count = df['원인'].value_counts()
category_count

부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# null인 값 확인 및 제거

In [18]:
df_bak = df.copy()

In [19]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df = df.fillna('N')

In [20]:
df.shape

(8529, 17)

In [21]:
category_count = df['대상 선정'].value_counts()
category_count

X    5517
O    3012
Name: 대상 선정, dtype: int64

In [22]:
category_count = df['구분'].value_counts()
category_count

N          5517
기능(커넥터)    1601
실내부품이음     1050
하체이음        361
Name: 구분, dtype: int64

In [23]:
category_count = df['책임구'].value_counts()
category_count

N       5517
구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [24]:
category_count = df['원인'].value_counts()
category_count

N        5517
부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# 전처리

In [24]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df['target_pred'] = df['대상 선정'].astype(str).copy()

In [25]:
df['target_pred'].head()

0    X
1    X
2    X
3    X
4    X
Name: target_pred, dtype: object

## 1. 트위터 형태소 분석

In [26]:
from ckonlpy.tag import Twitter
twitter = Twitter()

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [27]:
# user dictionary 등록
twitter.add_dictionary(['가니쉬','가스','가죽','간격','간격 단차','간섭','간헐적','감지 센서','개선품','개조','개폐','견인',
                       '경고등','경보음','고객','고장코드','고정','공기압','과다','구멍','그로멧','그리스','글라스','글라스런',
                       '글로브박스','기어','기준','꺾임','나사산','내부','내장재','냉각수','냉간','너트','네비게이션','노브',
                       '노즐','누기','누수','누유','누출','다이얼','단선','단자','단차','단품','닫힘','답력','대쉬','덕트',
                       '데이라이트','데코레이션','도색','도어','도어락','뒤','드레인','들뜸','떨림','떨어짐','라디에이터',
                       '라벨','라이닝','라이선스','라이선스 플레이트','락','란스','램프','랫치','러기지','런찬넬','레귤레이터',
                       '레그','레그타','레버','레벨','레벨링','레인지','레인프','레일','로워암','로컬','롤러','롤러암','루프',
                       '루프랙','룸램프','리모컨','리모트','리어','리어뷰','리저버','리클라이너','리턴','리테이너','리프터',
                       '리프팅','리플렉터','릴레이','릴리즈','링키지','마모','마운팅','마킹','매니폴드','매트','머드가드',
                       '머플러','멀티펑션','멈춤','멤버','모니터','모듈','모터','몰딩','무거움','무드','미결선','미러','미작동',
                       '미장착','미조립','미체결','바디','박음부','배터리','백워닝','백판넬','밸런스','밸브','버클','버튼',
                       '범퍼','베어링','베젤','벤틸레이션','벨트','벨트라인','변속','볼트','부밍음','부조','불규칙','불량',
                       '브라켓','브레이크','블랭킹','블레이드','블로우','블루','블루투스','블루핸즈','비상등','비포장','사이',
                       '사이드','사이드멤버','사제','삼각대','상승','상승 하강','생산','섀시','선글라스','선루프','선바이져',
                       '선쉐이드','세이프티','세차','센서','센터','소손','소켓','속도','송풍','송풍구','쇼버','쇽업','수동',
                       '쉘프','쉬라우드','쉬프트','스마트','스마트키','스위치','스캔','스커프','스크래치','스키드','스트라이커',
                       '스트러트','스티어링','스티어링휠','스포일러','스피커','슬라이딩','시거잭','시동꺼짐','시뮬레이션','시운전',
                       '시트','시트백','시트벨트','실내등','실러','실런트','쏠림','씰링','아울렛','아웃','아웃사이드',
                       '아웃사이드 미러','아이들','악셀','안개등','안테나','안티','암레스트','앞','액츄에이터','액티브',
                       '어라운드뷰','어셈블리','어스','어시스트','언더','언락','언폴딩','얼라인먼트','없음','에어백','에어벤트',
                       '에어컨','에이밍','엔진','엔진룸','엠블럼','여닫음','연동','연료','열림','열선','오버슬램','오버슬램범퍼',
                       '오버헤드','오장착','오체결','오결선','오토','오토큐','오토크루즈','와셔','와이어링','와이퍼','완체결',
                       '외부','요소수','요철','우','운전석','워셔액','원인품','웨더스트립','윈드실드','유닛','유막','유보',
                       '이격','이모빌라이져','이물질','이상','이음','이종품','익스텐션','인도','인버터','인사이드','인사이드 미러',
                       '인슐레이션','인스트','인젝터','인터쿨러','있음','자동','작동','작동 불량','작동력','작동음','잠금',
                       '장착','장치','재발','재부팅','재시동','재이탈','재장착','재조립','재체결','저항','전압','점검','점등',
                       '점멸','정렬','정션','정위치','정차','정체결','제동','제어','조명','조수석','조인트','조작감','조향각',
                       '좌','주름','주차','주행','진공','찬넬','천장','체결','체인지','체커','체크','쳐짐','충격','충전량',
                       '치핑','카울','카페트','캘리퍼','캠버','캡','커넥터','커버','커버링','커튼','커플러','컨트롤','컬럼',
                       '컴프레셔','컴플리트','컵홀더','케이블','코니시티','코드표','콘센트','콘솔','콘티넨탈','콤비','콤비램프',
                       '쿠션','쿨링팬','쿼터','퀵커넥터','크랙','크러쉬','크러쉬패드','크루즈','클램프','클러스터','클럭스프링',
                       '클리너','클립','타이어','탈거','탈부착','태핑스크류','터미널','턴램프','테스트','테이핑','테일게이트',
                       '텐션','토출','토크','통신','트라이앵글홀더','트랜스버스','트랜지스터','트렁크','트레이','트림','틀어짐',
                       '파워','파이프','파킹','파팅','판넬','패드','패들','패스너','패키지트레이','퍼스널','페달','페인트','펜더',
                       '펴짐','평탄로','포그램프','폴딩','풀리','풀핸들','풍절음','퓨즈','프런트','프레임','프로텍터','플랜지',
                       '플러그','플레이트','플로워','피더','픽싱','핀','필러','필러넥','필름','하강','하네스','하드웨어','하우징',
                       '하이드로릭','하이패스','해제','핸들','헤드라이닝','헤드라이트','헤드램프','헤드레스트','헤드업디스플레이',
                       '현상','협력사','호스','혼','홀더','확인','회로','회수량','후드','휘어짐','휠가드','휨','히터','힌지'],
                       'Noun')

In [28]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    #text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [29]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)

Wall time: 97.6 ms


In [30]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S','결과','공임','광주','넥쏘','니로','레이','렌트','리무진',
             '모닝','모하비','벨로스터','사진','상기','서비스','소하','스타렉스','스팅어','스포티지','신차','싼타페','써비스',
             '쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','언덕','엑센트','열','울산','임시 넘버','임시 번호','입고',
             '저머검내용','점검결과','조치결과','주차장','증상','지하','철도','첨부','첨부파일','청구','출고','카 마스터','카니발',
             '카렌스','코나','클레임','투싼','팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상',
             '현 상','현산','화성','그랜저','새벽','새차','야간','주간','그랜져','주재원','아침','밤, 심야','낮,낫','주제원']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [31]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 465 ms


In [32]:
# 형태소 분석기 
# twitter.pos()함수에다가 dataframe을 직접 대입하면 함수내 내장된 split함수에서 에러발생함 → 그래서 apply를 이용해야 함   
twitter = Twitter() 
def posextract(text):
    text = twitter.pos(text)
    return text

In [33]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(posextract)

Wall time: 23.4 s


In [34]:
type(df['clm_prtl_sbc'])

pandas.core.series.Series

In [35]:
df['clm_prtl_sbc'].head()

0    [(엔진, Noun), (룸, Noun), (다량, Noun), (의, Josa),...
1    [(리, Noun), (어, Suffix), (도어, Noun), (앤티, Noun...
2    [(커튼, Noun), (후크, Noun), (미장, Noun), (착, Noun)...
3    [(차량, Noun), (출고, Noun), (시, Noun), (웨즈, Noun)...
4    [(몰딩, Noun), (내부, Noun), (고무, Noun), (미조, Noun...
Name: clm_prtl_sbc, dtype: object

In [ ]:
stopwords = {'현상', '현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검원인','사진첨부','사진첨부','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F LH','F RH','L H','R H','L H리어',
             'R H리어','LH RH','LH','RH','RH','LH','RLH','R RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C 10000','C 100','좌','우','A S','결과','공임','광주','넥쏘','니로','레이','렌트','리무진',
             '모닝','모하비','벨로스터','사진','상기','서비스','소하','스타렉스','스팅어','스포티지','신차','싼타페','써비스',
             '쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','언덕','엑센트','열','울산','임시 넘버','임시 번호','입고',
             '저머검내용','점검결과','조치결과','주차장','증상','지하','철도','첨부','첨부파일','청구','출고','카 마스터','카니발',
             '카렌스','코나','클레임','투싼','팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상',
             '현 상','현산','화성','그랜저','새벽','새차','야간','주간','그랜져','주재원','아침','밤','심야','낮낫','주제원'}

replace = {'가니시:가니쉬','개스:가스','간극:간격','갭:간격','간단차:간격 단차','갭단차:간격 단차','단간차:간격 단차','간혈적:간헐적',
           '감지센서:감지 센서','울트라센서:감지 센서','울트라 센서:감지 센서','개페:개폐','개패:개폐','문구:고장코드','맨트:고장코드',
           '멘트:고장코드','고장 코드:고장코드','구먼:구멍','그로맷:그로멧','구리스:그리스','그라스:글라스','글래스:글라스',
           '창문:글라스','윈도우:글라스','유리:글라스','글래스런:글라스런','글라스 런:글라스런','글래스 런:글라스런',
           '글로우 박스:글로브박스','글로브 박스:글로브박스','글러브 박스:글로브박스','글러브박스:글로브박스','클로브박스:글로브박스',
           '꺽임:꺾임','네비:네비게이션','내비게이션:네비게이션','절손:단선','닫힐:닫힘','닫들:닫힘','닫힌:닫힘','닫을:닫힘',
           '대시:대쉬','데시:대쉬','데쉬:대쉬','도장:도색','도아:도어','도어록:도어락','도어 록:도어락','도어 락:도어락','들듬:들뜸',
           '라디에타:라디에이터','라디에디터:라디에이터','봉인:라이선스','라이센스:라이선스','번호판:라이선스 플레이트','록:락',
           '로크:락','록크:락','렘프:램프','전구:램프','래치:랫치','레치:랫치','러기쥐:러기지','런채널:런찬넬','런챈널:런찬넬',
           '런첸널:런찬넬','런체널:런찬넬','런잔넬:런찬넬','란찬넬:런찬넬','레규레이터:레귤레이터','레귤레터:레귤레이터',
           '레규레더:레귤레이터','유리기어:레귤레이터','래그:레그','레바:레버','레벌링:레벨링','레인진:레인지','로암:로워암',
           '로라:롤러','롤라:롤러','롤러 암:롤러암','루푸:루프','루푸 랙:루프랙','루푸렉:루프랙','룸 램프:룸램프','룸렘프:룸램프',
           '오버헤드 콘솔:룸램프','리모콘:리모컨','리모트 컨트롤:리모컨','리아:리어','리러:리어','리어 뷰:리어뷰','리프트:리프터',
           '마운틴:마운팅','마운트:마운팅','매니폴더:매니폴드','메트:매트','흙받이:머드가드','휠가드:머드가드','렉:멈춤','화면:모니터',
           '모튤:모듈','모타:모터','밀러:미러','미동작:미작동','미창작:미장착','보디:바디','밧데리:배터리','받데리:배터리',
           '베터리:배터리','빽워닝:백워닝','빽판넬:백판넬','바란스:밸런스','버턴:버튼','보튼:버튼','보턴:버튼','범버:범퍼','밤바:범퍼',
           '밸트:벨트','안됨:불량','않됨:불량','안내려감:불량','안올라감:불량','줄량:불량','브래이크:브레이크','브러쉬:블레이드',
           '부라쉬:블레이드','블레이크:블레이드','블로워:블로우','블로어:블로우','불루:블루','상향:상승','올라감:상승',
           '상하강:상승 하강','상하:상승 하강','샤시:섀시','썬글라스:선글라스','선글래스:선글라스','썬글래스:선글라스','썬루프:선루프',
           '선루푸:선루프','썬바이저:선바이져','선바이저:선바이져','썬바이져:선바이져','화장:선바이져','선세이드:선쉐이드',
           '썬쉐이드:선쉐이드','썬세이드:선쉐이드','쎄이프티:세이프티','중앙:센터','벨로시터:속도','벨로시티:속도','통풍:송풍',
           '토출구,통풍구:송풍구','쇼바:쇼버','소버:쇼버','쇼업:쇽업','시프트:쉬프트','포브키:스마트키','스마트 키:스마트키',
           '스케너:스캔','스캐너:스캔','스켄:스캔','스캔너:스캔','스켄너:스캔','기스:스크래치','스크레치:스크래치','스크러치:스크래치',
           '스크럿치:스크래치','스트라이크:스트라이커','스타라이커:스트라이커','스티어링 휠:스티어링휠','스포이러:스포일러',
           '스라이딩:슬라이딩','슬라이징:슬라이딩','슬라이닝:슬라이딩','시가잭:시거잭','시동 꺼짐:시동꺼짐','시몰레이션:시뮬레이션',
           '씨트:시트','시트 백:시트백','시트빽:시트백','시트 빽:시트백','시트 벨트:시트벨트','시트 밸트:시트벨트','안전밸트:시트벨트',
           '안전벨트:시트벨트','안전 벨트:시트벨트','안전 밸트:시트벨트','실레등:실내등','씰러:실러','실링:씰링','아우터:아웃',
           '아웃트:아웃','아웃:아웃사이드','아우터:아웃사이드','아웃트:아웃사이드','백미러:아웃사이드 미러','빽미러:아웃사이드 미러',
           '빽밀러:아웃사이드 미러','폴딩 미러:아웃사이드 미러','아웃사이드 리어뷰 미러:아웃사이드 미러','사이드 미러:아웃사이드 미러',
           '앗셀:악셀','앤티:안티','앤티:안티','안티노이즈:안티','액추에이터:액츄에이터','엑츄레이터:액츄에이터','서랴운드부:어라운드뷰',
           '서라운드 뷰:어라운드뷰','앗세이:어셈블리','어셈블:어셈블리','어셉블리:어셈블리','어스스트:어시스트','언록:언락',
           '언로크:언락','언록크:언락','펴짐:언폴딩','얼라이먼트:얼라인먼트','없는:없음','없슴:없음','멊음:없음','에어콘:에어컨',
           '엠블렘:엠블럼','앰블램:엠블럼','연계:연동','퓨얼:연료','휴얼:연료','연료필러:연료','열릴:열림','열을:열림','개방:열림',
           '오픈:열림','오버 슬램:오버슬램','오버슬램:오버슬램','슬랩:오버슬램','오버슬램 댐퍼:오버슬램범퍼',
           '오버슬램 범퍼:오버슬램범퍼','와샤:와셔','와이어:와이어링','드라이브:운전석','와셔액:워셔액','와샤액:워셔액',
           '웨즈스트립:웨더스트립','웨져스터립:웨더스트립','웨져스트립:웨더스트립','웨져:웨더스트립','웨더:웨더스트립','웨쟈:웨더스트립',
           '이모:이모빌라이져','이모빌라이저:이모빌라이져','소음:이음','노이즈:이음','진동음:이음','공명음:이음','소리:이음',
           '인버트:인버터','이너:인사이드','인너:인사이드','룸미러:인사이드 미러','룸밀러:인사이드 미러',
           '리어뷰 인사이드 미러:인사이드 미러','인슐레이터:인슐레이션','인슐레이트:인슐레이션','있는:있음','있슴:있음','갇동:작동',
           '작불:작동 불량','작동불량:작동 불량','작역:작동력','동작음:작동음','창작:장착','창치:장치','재발생:재발','제발생:재발',
           '제시동:재시동','제장착:재장착','제조립:재조립','재조임:재체결','제체결:재체결','통전:전압','정열:정렬','정선:정션',
           '정채결:정체결','동승석:조수석','페시브:조수석','패시브:조수석','채널:찬넬','챈널:찬넬','첸널:찬넬','체널:찬넬','잔넬:찬넬',
           '조임:체결','체격:체결','취핑:치핑','취밍:치핑','피칭:치핑','카울 탑:카울','카울 톱:카울','카매트:카페트','카 매트:카페트',
           '카메트:카페트','카 메트:카페트','카펫:카페트','캡머:캠버','켑:캡','뚜껑:캡','커낵터:커넥터','컨넥:커넥터','컨넥터:커넥터',
           '커텐:커튼','커덴:커튼','카풀러:커플러','커풀러:커플러','콘트롤:컨트롤','콘드롤:컨트롤','콘트롤러:컨트롤','컨트롤러:컨트롤',
           '칼럼:컬럼','컴프레샤:컴프레셔','콤프레샤:컴프레셔','콤푸레셔:컴프레셔','콤푸레샤:컴프레셔','콤프레셔:컴프레셔',
           '콤프:컴프레셔','컵 홀더:컵홀더','케불:케이블','컨센트:콘센트','센터페시아:콘솔','센타페시아:콘솔','큰티넨탈:콘티넨탈',
           '컴비네이션:콤비','컴비:콤비','후진등:콤비램프','후미등:콤비램프','컴비램프:콤비램프','컴비네이션 램프:콤비램프',
           '콤비네이션 램프:콤비램프','쿠쉬:쿠션','쿠시:쿠션','쿳션:쿠션','쿨링펜:쿨링팬','쿨링휀:쿨링팬','퀵컨넥터:퀵커넥터',
           '퀵커낵터:퀵커넥터','퀵컨넥:퀵커넥터','크릭:크랙','크래쉬:크러쉬','크래쉬패드:크러쉬패드','크러쉬 패드:크러쉬패드',
           '크래쉬 패드:크러쉬패드','크램프:클램프','계기판:클러스터','클러스트:클러스터','클락스프링:클럭스프링',
           '클랄스프링:클럭스프링','크리너:클리너','바퀴:타이어','터미날:터미널','테잎:테이핑','백도어:테일게이트','빽도어:테일게이트',
           '테일:테일게이트','트렌지스터:트랜지스터','트렁크리드:트렁크','트렁크도어:트렁크','틀레이:트레이','트레일:트레이','트립:트림',
           '파웜:파워','패널:판넬','펜널:판넬','화스너:패스너','패키지 트레이:패키지트레이','퍼스날:퍼스널','폐달:페달','패달:페달',
           '페이트:페인트','휀더:펜더','휀다:펜더','핸더:펜더','훼더:펜더','헨다:펜더','포그 램프:포그램프','접이:폴딩','접힘:폴딩',
           '휴즈, 휴쥬:퓨즈','프론트:프런트','후런트:프런트','후론트:프런트','플레임:프레임','플렌지:플랜지','플로그:플러그',
           '프로그:플러그','플로아:플로워','플로어:플로워','필라:필러','필럼:필러','필라넥:필러넥','필림:필름','하향:하강',
           '내려감:하강','하니스:하네스','하이페스:하이패스','헤제:해제','헨들:핸들','핸드:핸들','해드라이닝:헤드라이닝',
           '해트라이닝:헤드라이닝','해드라이트:헤드라이트','헤드 램프:헤드램프','해드램프:헤드램프','해드 램프:헤드램프',
           '해드업디스플레이:헤드업디스플레이','업체:협력사','크락션:혼','클락션:혼','후크:홀더','훗:홀더','훅:홀더','후크:홀더',
           '본네트:후드','본넷:후드','구부러짐:휨','구브러짐:휨','히트:히터','히타:히터'}

In [54]:
from ckonlpy.tag import Postprocessor

## 파일을 로딩해서 전처리 하기 ##
from ckonlpy.utils import load_wordset
# passwords = load_wordset('./passwords.txt')
stopwords = load_wordset('./stops.txt')

from ckonlpy.utils import load_replace_wordpair
replace = load_replace_wordpair('./replace.txt')

from ckonlpy.utils import load_ngram
# ngrams = load_ngram('./ngrams.txt')
####################################

# passwords = passwords, # 해당 단어만 선택
# passtags = passtags, # 해당 품사만 선택
# ngrams = ngrams # 해당 복합 단어 set을 한 단어로 결합

# print(passwords)
print(stopwrods)
print(replace)
# print(ngrams)


NameError: name 'stopwrods' is not defined

In [42]:
def postprocessor(text):
    postprocessor = Postprocessor(
    base_tagger = twitter,  # base tagger
    stopwords = stopwords,  # 해당 단어 필터
    replace = replace)      # 해당 단어 set 치환
    text = postprocessor(text)
    return text

In [37]:
# df['clm_prtl_sbc'] = df['clm_prtl_sbc'].astype(str).copy()

In [43]:
post_result = df['clm_prtl_sbc'].apply(postprocessor)

TypeError: 'Postprocessor' object is not callable

In [48]:
post_result

[("[('", 'Punctuation'),
 ("',", 'Punctuation'),
 ("'", 'Punctuation'),
 ('Alpha', 'Alpha'),
 ("'),", 'Punctuation'),
 ("('", 'Punctuation'),
 ("',", 'Punctuation'),
 ("'", 'Punctuation'),
 ('Noun', 'Alpha'),
 ("'),", 'Punctuation'),
 ("('", 'Punctuation'),
 ('리', 'Noun'),
 ("',", 'Punctuation'),
 ("'", 'Punctuation'),
 ('Noun', 'Alpha'),
 ("'),", 'Punctuation'),
 ("('", 'Punctuation'),
 ('어', 'Eomi'),
 ("',", 'Punctuation'),
 ("'", 'Punctuation'),
 ('Suffix', 'Alpha'),
 ("'),", 'Punctuation'),
 ("('", 'Punctuation'),
 ('도어', 'Noun'),
 ("',", 'Punctuation'),
 ("'", 'Punctuation'),
 ('Noun', 'Alpha'),
 ("'),", 'Punctuation'),
 ("('", 'Punctuation'),
 ('앤티', 'Noun'),
 ("',", 'Punctuation'),
 ("'", 'Punctuation'),
 ('Noun', 'Alpha'),
 ("'),", 'Punctuation'),
 ("('", 'Punctuation'),
 ('취핑', 'Noun'),
 ("',", 'Punctuation'),
 ("'", 'Punctuation'),
 ('Noun', 'Alpha'),
 ("'),", 'Punctuation'),
 ("('", 'Punctuation'),
 ('필림', 'Noun'),
 ("',", 'Punctuation'),
 ("'", 'Punctuation'),
 ('Noun', 'Al

In [44]:
pos_result[5]

"[('C', 'Alpha'), ('현상', 'Noun'), ('리', 'Noun'), ('어', 'Suffix'), ('윈도우', 'Noun'), ('판넬', 'Noun'), ('트립', 'Noun'), ('미장', 'Noun'), ('착', 'Noun'), ('내용', 'Noun'), ('차량', 'Noun'), ('출고', 'Noun'), ('후', 'Noun'), ('리', 'Noun'), ('어', 'Suffix'), ('윈도우', 'Noun'), ('판넬', 'Noun'), ('트립', 'Noun'), ('좌측', 'Noun'), ('미장', 'Noun'), ('착', 'Noun'), ('됨', 'Verb'), ('조치', 'Noun'), ('리', 'Noun'), ('어', 'Suffix'), ('윈도우', 'Noun'), ('판넬', 'Noun'), ('트립', 'Noun'), ('좌측', 'Noun'), ('조립', 'Noun')]"

## 2. 트위터 명사추출 및 불용어 제거 

In [34]:
#명사 추출 함수
def nounextract(text):
    text = Okt.nouns(text)
    return text

In [35]:
df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(nounextract)

In [36]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    #text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [37]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S','결과','공임','광주','넥쏘','니로','레이','렌트','리무진',
             '모닝','모하비','벨로스터','사진','상기','서비스','소하','스타렉스','스팅어','스포티지','신차','싼타페','써비스',
             '쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','언덕','엑센트','열','울산','임시 넘버','임시 번호','입고',
             '저머검내용','점검결과','조치결과','주차장','증상','지하','철도','첨부','첨부파일','청구','출고','카 마스터','카니발',
             '카렌스','코나','클레임','투싼','팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상',
             '현 상','현산','화성','그랜저','새벽','새차','야간','주간','그랜져','주재원','아침','밤, 심야','낮,낫','주제원']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [38]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)

Wall time: 247 ms


Wall time: 1.48 s


## 3. Typo수정

In [55]:
df['clm_prtl_sbc'].head()

0    [(엔진, Noun), (룸, Noun), (다량, Noun), (의, Josa),...
1    [(리, Noun), (어, Suffix), (도어, Noun), (앤티, Noun...
2    [(커튼, Noun), (후크, Noun), (미장, Noun), (착, Noun)...
3    [(차량, Noun), (출고, Noun), (시, Noun), (웨즈, Noun)...
4    [(몰딩, Noun), (내부, Noun), (고무, Noun), (미조, Noun...
Name: clm_prtl_sbc, dtype: object

In [56]:
df['clm_prtl_sbc'].shape

(8529,)

In [62]:
df = df.reindex()

In [63]:
df.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   
3               W  2018     20180928         ZZ7         B11  조립 문제_미장착/오장착   
4               W  2016     20160526         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   
3         KV11          YP     A9          TT        83120   
4         KV11          YP     A9          TT        83860   

                                        clm_prtl_sbc 대상 선정 구분 책임구 원인  검토결과  \
0  [(엔진, Noun), (룸, Noun), (다량, Noun), (의, Josa),...     X  N   N  N  X---   
1  [(리, Noun), (어, Suffix), (도어, Noun), (앤티, Noun...     X  N   N  N  X---   
2  [(커튼, Noun), (후크, Noun), (미장, Noun), (착, Noun)...     X  N   N  N  X---   
3  [(차량, Noun), (출고, Noun), (시, Noun), (웨즈, Noun)...     X  N   N  N  X---   
4  [(몰딩, Noun), (내부, Noun), (고무, Noun), (미조, Noun...     X  N   N  N  X---   

  target_pred  
0           X  
1           X  
2           X  
3           X  
4           X

In [64]:
df[df['clm_prtl_sbc'].str.contains(userdic.missprint[1])].head()

KeyError: '[nan nan nan ... nan nan nan] not in index'

In [65]:
# 붙어 있는 단어 오자 검색후 replace 수정
index1=0
index2=0
data = df['clm_prtl_sbc']
for index2 in range(0,656):
    for index1 in range(0,8529):
        if (userdic.missprint[index2] == data[index1].str.contains(userdic.missprint[index2])):
            vocab[index1] = vocab[index1].replace(userdic.misspring[index2],userdic.standard[index2], True)
            index2+=1
        else:
            index1+=1

AttributeError: 'list' object has no attribute 'str'

In [ ]:
# 낱개 단어 오자 수정
index1=0
index2=0
for index2 in range(0,656):
    for index1 in range(0,8529):
        if (userdic.missprint[index2] == vocab[index1]):
            vocab[index1] = vocab[index1].replace(userdic.missprint[index2],userdic.standard[index2], True)            
            index2+=1
        else:
            index1+=1

In [40]:
# n_rows = len(df.index)
# target = df['clm_prtl_sbc']
# index = list(df.index)
# i=0
# for i in range(n_rows):
#     choice = index[i]
#     userdic_stop = usedic['User Dic.'][0:choice]
#     userdic_synom = usedic['동의어'][0:choice]
#     if (target.str.contains(userdic_stop[choice]) == userdic_stop[choice]):
#         df.replace(to_replace = userdic_stop ,value = userdic_synom)
#     else : 
#         i = i + 1 

In [41]:
# df2 = df[df['clm_prtl_sbc'].str.contains(usedic.동의어[1])]
# df2['clm_prtl_sbc'].replace(to_replace='개스',value='가스')

In [42]:
# df['clm_prtl_sbc'] = df2['clm_prtl_sbc'].replace(to_replace='간극',value='간격')

In [43]:
# df1 = df.replace(to_replace="가니시",value="가니쉬")
# df1 = df.replace(to_replace="개스",value="가스")
# df1 = df.replace(to_replace="간극",value="간격")
# df1 = df.replace(to_replace="간단차",value="간격 단차")
# df1 = df.replace(to_replace="갭단차",value="간격 단차")
# df1 = df.replace(to_replace="단간차",value="간격 단차")
# df1 = df.replace(to_replace="간혈적",value="간헐적")
# df1 = df.replace(to_replace="감지센서",value="감지 센서")
# df1 = df.replace(to_replace="울트라센서",value="감지 센서")
# df1 = df.replace(to_replace="울트라 센서",value="감지 센서")
# df1 = df.replace(to_replace="개페",value="개폐")
# df1 = df.replace(to_replace="개패",value="개폐")
# df1 = df.replace(to_replace="문구",value="고장코드")
# df1 = df.replace(to_replace="맨트",value="고장코드")
# df1 = df.replace(to_replace="멘트",value="고장코드")

# 학습데이터와 테스트데이터 만들기

In [44]:
df = df.reindex() #index 재설정. reindex:새로운 색인에 맞도록 객체를 새로 생성하는 기능.

In [45]:
df.shape

(8529, 18)

In [46]:
from sklearn.model_selection import train_test_split

In [47]:
df_train_v,df_test_v = train_test_split(df, test_size = 0.1, random_state=123)

In [48]:
df_train = df_train_v.copy()
df_test = df_test_v.copy()

In [49]:
df_train.shape

(7676, 18)

In [50]:
df_test.shape

(853, 18)

# 단어 벡터화

In [51]:
stop_dict = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S','결과','공임','광주','넥쏘','니로','레이','렌트','리무진',
             '모닝','모하비','벨로스터','사진','상기','서비스','소하','스타렉스','스팅어','스포티지','신차','싼타페','써비스',
             '쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','언덕','엑센트','열','울산','임시 넘버','임시 번호','입고',
             '저머검내용','점검결과','조치결과','주차장','증상','지하','철도','첨부','첨부파일','청구','출고','카 마스터','카니발',
             '카렌스','코나','클레임','투싼','팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상',
             '현 상','현산','화성','그랜저','새벽','새차','야간','주간','그랜져','주재원','아침','밤, 심야','낮,낫','주제원']

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = stop_dict, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 1, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 20000 # 만들 피처의 수, 단어의 수가 된다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None,
        stop_words=['현상', '[C]', '현상및점검', '현    상', '현 상', '점검점검', '점검내용', '조치내용', '점검점검내용', '현상및조치', 'C', '원인', '점검/원인', '사진첨부', '(사진첨부)', '사진참조', '점검및원인', '동영상참조', '동영상', '대리점', '고객', '고객에게', '상기', '차량', '상기차량은', '자동차', '당차량은', '당차량', '당', '차량은', '점검및조치내용', 'F/LH', 'F/RH', 'L/H', 'R/H', 'L/H리어', 'R/H리어', ...'현  상', '현 상', '현산', '화성', '그랜저', '새벽', '새차', '야간', '주간', '그랜져', '주재원', '아침', '밤, 심야', '낮,낫', '주제원'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [53]:
#%%time 
train_feature_vector = vectorizer.fit_transform(df_train['clm_prtl_sbc'])
train_feature_vector.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['100', '10000', 'h리어', 'lh', 'lh쪽', 'rh', 'rh쪽', '넘버', '리어lh', '리어rh', '마스터', '번호', '심야', '임시', '점검'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(7676, 20000)

In [54]:
#%%time 
test_feature_vector = vectorizer.fit_transform(df_test['clm_prtl_sbc'])
test_feature_vector.shape

(853, 20000)

In [55]:
type(train_feature_vector)

scipy.sparse.csr.csr_matrix

In [56]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

20000


['가감',
 '가감 실내',
 '가감 실내 대쉬보드',
 '가감 저속',
 '가감 저속 등판',
 '가끔',
 '가끔 소음',
 '가끔 소음 허브',
 '가끔 재현',
 '가끔 재현 발생']

In [57]:
len(vocab)

20000

In [58]:
userdic.missprint[0]

'가니시'

In [59]:
userdic.standard[0]

'가니쉬'

In [60]:
# 낱개 단어 오자 수정
index1=0
index2=0
for index2 in range(0,656):
    for index1 in range(0,20000):
        if (userdic.missprint[index2] == vocab[index1]):
            vocab[index1] = vocab[index1].replace(userdic.missprint[index2],userdic.standard[index2], True)            
            index2+=1
        else:
            index1+=1       

In [ ]:
# 붙어 있는 단어 오자 수정
index1=0
index2=0
for index2 in range(0,656):
    for index1 in range(0,20000):
        if (userdic.missprint[index2] == vocab[index1].str.contains(usedic.missprint[index2])):
            vocab[index1] = vocab[index1].replace(userdic.misspring[index2],userdic.standard[index2], True)
            index2+=1
        else:
            index1+=1

In [63]:
vocab.str.contains(usedic.missprint[0])

AttributeError: 'list' object has no attribute 'str'

In [64]:
type(vocab)

list

In [184]:
vocab[300:400]

['감형',
 '감형 소음',
 '감형 소음 발생',
 '강변',
 '강변 도로',
 '강변 도로 부근',
 '강시',
 '강시 마찰',
 '강시 마찰 소음',
 '강시 소음',
 '강시 소음 발생',
 '강시 윈도우',
 '강시 윈도우 몰딩',
 '강제',
 '강제 동시',
 '강제 동시 작동',
 '강제 동시 정상',
 '강제 손상',
 '강제 손상 추정',
 '강제 작동',
 '강제 작동 하나',
 '강제 재생',
 '강제 재생 차압',
 '강제 출력',
 '강제 출력 제한',
 '강제 케이블',
 '강제 케이블 풀림',
 '강제 헤드',
 '강제 헤드 램프',
 '개도',
 '개도 목표',
 '개도 목표 이상',
 '개선',
 '개선 고속',
 '개선 고속 주행',
 '개선 교환',
 '개선 불량',
 '개선 불량 교환',
 '개선 사용',
 '개선 사용 교환',
 '개선 요망',
 '개선 요망 사항',
 '개선 요망 임시',
 '개선 운전',
 '개선 운전 밸트라',
 '개선 작업',
 '개선 작업 완료',
 '개선 조속',
 '개선 조속 개선',
 '개선 조치',
 '개선 조치 내용',
 '개선 천장',
 '개선 천장 헤드',
 '개선 필요',
 '개선 확인',
 '개선 휠바',
 '개선 휠바 란스',
 '개조',
 '개조 부분',
 '개조 부분 조치',
 '개조 부품',
 '개조 부품 흔적',
 '개조 분해',
 '개조 분해 흔적',
 '개조 사항',
 '개조 사항 센터',
 '개조 사항 컨트롤',
 '개조 유무',
 '개조 유무 점검',
 '개조 흔적',
 '개조 흔적 점검',
 '개페시',
 '개페시 소음',
 '개페시 소음 발생',
 '개페시 질감',
 '개페시 질감 조치',
 '개폐',
 '개폐 경고',
 '개폐 경고 저음',
 '개폐 고부',
 '개폐 고부 저음',
 '개폐 끄르륵',
 '개폐 끄르륵 끄르륵',
 '개폐 동시',
 '개폐 동시 이음',
 '개폐 레일',
 '개폐 레일 커브',
 '개폐 미터',
 '개폐 미터 표시',
 '개

In [173]:
vocab[170] = vocab[170].replace(userdic.missprint[3],userdic.standard[3], True)
vocab[170]

'간격'

In [175]:
vocab[1] == userdic.missprint[2]

False

In [98]:
dist = np.sum(train_feature_vector, axis=0)
dist.shape

(1, 20000)

In [99]:
dist_pd = pd.DataFrame(dist, columns=vocab)
dist_pd

가감  가감 실내  가감 실내 대쉬보드  가감 저속  가감 저속 등판  가끔  가끔 소음  가끔 소음 허브  가끔 재현  \
0   5     10           3      5         3  15      4         4    199   

   가끔 재현 발생    ...     힌지 볼트  힌지 볼트 볼트  힌지 부위  힌지 부위 수석  힌지 스톱  힌지 스톱 볼트  \
0         4    ...         3         9      5         5      6        38   

   힌지 스트라이크  힌지 스트라이크 조정  힌지 조정  힌지 조정 작업  
0         3            3      5         5  

[1 rows x 20000 columns]

In [100]:
dist_pd.columns

Index(['가감', '가감 실내', '가감 실내 대쉬보드', '가감 저속', '가감 저속 등판', '가끔', '가끔 소음',
       '가끔 소음 허브', '가끔 재현', '가끔 재현 발생',
       ...
       '힌지 볼트', '힌지 볼트 볼트', '힌지 부위', '힌지 부위 수석', '힌지 스톱', '힌지 스톱 볼트',
       '힌지 스트라이크', '힌지 스트라이크 조정', '힌지 조정', '힌지 조정 작업'],
      dtype='object', length=20000)

In [101]:
dist_pd_t = dist_pd.T
dist_pd_t.head()

0
가감           5
가감 실내       10
가감 실내 대쉬보드   3
가감 저속        5
가감 저속 등판     3

In [102]:
dist_pd_t.index.name = "word"

In [103]:
dist_pd_t[:2]

0
word     
가감      5
가감 실내  10

In [104]:
dist_pd_t.reset_index(level='word', inplace=True)

In [105]:
dist_pd_t[:3]

word   0
0          가감   5
1       가감 실내  10
2  가감 실내 대쉬보드   3

In [106]:
dist_pd_t.rename(columns={0:'count'}, inplace = True)

In [107]:
dist_pd_t[:200]

word  count
0            가감      5
1         가감 실내     10
2    가감 실내 대쉬보드      3
3         가감 저속      5
4      가감 저속 등판      3
5            가끔     15
6         가끔 소음      4
7      가끔 소음 허브      4
8         가끔 재현    199
9      가끔 재현 발생      4
10           가나      3
11        가나 왼쪽     39
12     가나 왼쪽 소리      3
13       가나 음으로      6
14    가나 음으로 스티      4
15           가능      3
16        가능 거리      6
17     가능 거리 달라      3
18     가능 거리 변동      3
19        가능 도어      6
20     가능 도어 인너      3
21          가니쉬      3
22       가니쉬 고정      3
23    가니쉬 고정 불량      4
24       가니쉬 고함      3
25   가니쉬 고함 가니쉬      3
26       가니쉬 교체     15
27       가니쉬 교환      4
28    가니쉬 교환 정상      7
29    가니쉬 교환 조치      4
..          ...    ...
170          간극      3
171       간극 범퍼      8
172    간극 범퍼 본넷      5
173       간극 불량     49
174    간극 불량 소음      3
175    간극 불량 점검      4
176    간극 불량 조정      3
177       간극 아래      3
178    간극 아래 범퍼      3
179       간극 조정      8
180    간극 조정 시운      5
181   간극 조정 시운전     23
182    간극 조정 우측      3
183    간극 조정 타이     12
184       간극 조치     47
185    간극 조치 내용      4
186      간극 협소하      3
187   간극 협소하 도어     15
188       간극 확인      3
189    간극 확인 운전      4
190       간극 후드      3
191   간극 후드 앞범퍼      5
192          간단      5
193       간단 조정     14
194    간단 조정 작업      5
195          간섭      3
196       간섭 가속      4
197    간섭 가속 소음     32
198       간섭 도어     11
199    간섭 도어 도장      8

[200 rows x 2 columns]

In [260]:
dist_pd_t = dist_pd_t.replace(to_replace="간극",value="간격")
df1 = df.replace(to_replace="개스",value="가스")
df1 = df.replace(to_replace="간극",value="간격")
df1 = df.replace(to_replace="간단차",value="간격 단차")
df1 = df.replace(to_replace="갭단차",value="간격 단차")
df1 = df.replace(to_replace="단간차",value="간격 단차")
df1 = df.replace(to_replace="간혈적",value="간헐적")
df1 = df.replace(to_replace="감지센서",value="감지 센서")
df1 = df.replace(to_replace="울트라센서",value="감지 센서")
df1 = df.replace(to_replace="울트라 센서",value="감지 센서")
df1 = df.replace(to_replace="개페",value="개폐")
df1 = df.replace(to_replace="개패",value="개폐")
df1 = df.replace(to_replace="문구",value="고장코드")
df1 = df.replace(to_replace="맨트",value="고장코드")
df1 = df.replace(to_replace="멘트",value="고장코드")

In [470]:
aa = vectorizer.fit_transform(dist_pd_t['word'])

In [472]:
aa.shape

(20000, 20000)

In [261]:
dist_pd_t[:200]

word  count
0            가감      5
1         가감 실내     10
2    가감 실내 대쉬보드      3
3         가감 저속      5
4      가감 저속 등판      3
5            가끔     15
6         가끔 소음      4
7      가끔 소음 허브      4
8         가끔 재현    199
9      가끔 재현 발생      4
10           가나      3
11        가나 왼쪽     39
12     가나 왼쪽 소리      3
13       가나 음으로      6
14    가나 음으로 스티      4
15           가능      3
16        가능 거리      6
17     가능 거리 달라      3
18     가능 거리 변동      3
19        가능 도어      6
20     가능 도어 인너      3
21          가니쉬      3
22       가니쉬 고정      3
23    가니쉬 고정 불량      4
24       가니쉬 고함      3
25   가니쉬 고함 가니쉬      3
26       가니쉬 교체     15
27       가니쉬 교환      4
28    가니쉬 교환 정상      7
29    가니쉬 교환 조치      4
..          ...    ...
170          간격      3
171       간극 범퍼      8
172    간극 범퍼 본넷      5
173       간극 불량     49
174    간극 불량 소음      3
175    간극 불량 점검      4
176    간극 불량 조정      3
177       간극 아래      3
178    간극 아래 범퍼      3
179       간극 조정      8
180    간극 조정 시운      5
181   간극 조정 시운전     23
182    간극 조정 우측      3
183    간극 조정 타이     12
184       간극 조치     47
185    간극 조치 내용      4
186      간극 협소하      3
187   간극 협소하 도어     15
188       간극 확인      3
189    간극 확인 운전      4
190       간극 후드      3
191   간극 후드 앞범퍼      5
192          간단      5
193       간단 조정     14
194    간단 조정 작업      5
195          간섭      3
196       간섭 가속      4
197    간섭 가속 소음     32
198       간섭 도어     11
199    간섭 도어 도장      8

[200 rows x 2 columns]

# tf-idf 가중치 적용

In [48]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [49]:
#%%time 
train_feature_tfidf = transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

(7676, 20000)

In [50]:
#%%time 
test_feature_tfidf = transformer.fit_transform(test_feature_vector)
test_feature_tfidf.shape

(853, 20000)

# 머신러닝 : 학습

In [51]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [52]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
y_label = df_train['target_pred']
# y_label3 = df_train['charge_pred']
%time forest = forest.fit(train_feature_tfidf, y_label)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 1.63 s


# 모델평가

In [53]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, train_feature_vector, y_label, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 27.3 s


array([0.80078125, 0.80781759, 0.78501629, 0.80065147, 0.80325733])

In [54]:
round(np.mean(score)*100,2)

79.95

# 예측

In [55]:
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(test_feature_vector)
y_pred[:3]

array(['X', 'X', 'X'], dtype=object)

In [56]:
y_pred.shape

(853,)

In [57]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output = pd.DataFrame(data={'category_pred':y_pred})
output.head(10)

category_pred
0             X
1             X
2             X
3             X
4             X
5             X
6             X
7             X
8             X
9             X

In [58]:
df_test['category_pred'] = y_pred

# 결과검증

In [59]:
df_test['pred_diff'] = 0
df_test['pred_diff'] = (df_test['대상 선정'] == df_test['category_pred'] ) == 1
df_test['pred_diff'] = df_test['pred_diff'].astype(int)
df_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd  \
124                W  2015     20151103         ZZ7         B11   
1073               W  2017     20170828         ZZ7         B14   
3054               W  2016     20160906         ZZ5         I12   
1527               W  2018     20180831         ZZ7         B17   
867                W  2018     20180905         ZZ7         B13   

                ceq_psit_nm trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd  \
124           조립 문제_미장착/오장착         KV11          YP     A9          TT   
1073             조립 문제_간/단차         KV11          YP     A9          CC   
3054  작동불량_작동이 원활하지 않음(무거움)         KV11          YP     A9          TT   
1527           조립 문제_정위치 불량         KV11          CK     J5          AC   
867             조립 문제_유격/유동         KV11          YP     A9          CC   

     repn_part_cd                                       clm_prtl_sbc 대상 선정  \
124         85765       러기    네트    훅    미장    착    스    크류    적도...     X   
1073        12441       리    어도    발생    조치    리    휀더    판넬    스...     X   
3054        83362    리    어도    커튼    문제    점검    결과    커튼    오른쪽...     X   
1527        95811       후    측방    고등    표    점검    내용    및    조치...     X   
867         56115       주행    휠    보스    유격    발생    점검    및     ...     O   

          구분   책임구    원인                검토결과 target_pred category_pred  \
124        N     N     N                X---           X             X   
1073       N     N     N                X---           X             X   
3054       N     N     N                X---           X             X   
1527       N     N     N                X---           X             X   
867   실내부품이음  생산공장  조립불량  O-실내부품이음-생산공장-조립불량           O             X   

      pred_diff  
124           1  
1073          1  
3054          1  
1527          1  
867           0

In [60]:
diff_sum = df_test['pred_diff'].sum()
diff_sum

577

In [61]:
y_pred_len = len(y_pred)
y_pred_len

853

# 정확도

In [62]:
Acy = round(diff_sum/y_pred_len*100,1)
Acy

67.6